# Creating a system LLM_based-on Tutor
This is a technial tutor that use GPT-4o to answer questions in a step-by-step manner. It is based on the Tutor LLM model.

In [5]:
# Import libraries
import os
import json
import string
import requests
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown, update_display
from openai import OpenAI

import datetime
print(f"This notebook was last run on {datetime.datetime.now():%Y-%m-%d %H:%M:%S}")

This notebook was last run on 2025-09-17 17:07:06


## Configure the environment parameters

In [6]:
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"

In [9]:
# Load model
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("The API Key is in right format!")
else:
    print("There's a problem in API key!!!")
    
openao = OpenAI()

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

The API Key is in right format!


In [ ]:
# here is the question; type over this to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [11]:
def messages_for(question, system_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

In [10]:
system_prompt = "You are a helpful Python teacher. Especially, you have a nice mindset in coding skills."
user_prompt = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [12]:
# Streaming response from gpt-4o-mini
client = OpenAI()
with client.chat.completions.stream(
    model="gpt-4o-mini",
    messages=messages_for(system_prompt, user_prompt),
) as stream:
    for event in stream:
        if event.type == "message.delta":
            # Print new text as it streams in
            print(event.delta, end="", flush=True)
        elif event.type == "message.completed":
            print("\n--- done ---")